In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.DataFrame({
   ....:     'x0': [1, 2, 3, 4, 5],
   ....:     'x1': [0.01, -0.01, 0.25, -4.1, 0.],
   ....:     'y': [-1.5, 0., 3.6, 1.3, -2.]})

In [3]:
data.values

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

In [4]:
d3 = data.copy()

In [5]:
d3['string'] = ['a','b','c','d','e']

In [6]:
d3.values

array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, 0.25, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

In [7]:
model_cols = ['x0', 'x1']
data.loc[:, model_cols].values

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

In [8]:
data['category'] = pd.Categorical(['a', 'b', 'a', 'a', 'b'],
   ....:                                   categories=['a', 'b'])
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


In [9]:
data_with_dummies = pd.get_dummies(data.category,prefix='Categories')

In [10]:
final_data = data.drop('category',axis=1).join(data_with_dummies)

In [11]:
final_data

,x0,x1,y,Categories_a,Categories_b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,-4.10,1.3,1,0
4,5,0.00,-2.0,0,1


### Creating Model Descriptions with Patsy

In [12]:
data = pd.DataFrame({
   ....:     'x0': [1, 2, 3, 4, 5],
   ....:     'x1': [0.01, -0.01, 0.25, -4.1, 0.],
   ....:     'y': [-1.5, 0., 3.6, 1.3, -2.]})

In [13]:
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [14]:
import patsy

In [15]:
y, X = patsy.dmatrices('y ~ x0 + x1', data)

In [16]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [17]:
np.asarray(y)

array([[-1.5],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2. ]])

In [18]:
np.asarray(X)

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

In [19]:
patsy.dmatrices('y ~ x0 + x1 + 0', data)

(DesignMatrix with shape (5, 1)
      y
   -1.5
    0.0
    3.6
    1.3
   -2.0
   Terms:
     'y' (column 0),
 DesignMatrix with shape (5, 2)
   x0     x1
    1   0.01
    2  -0.01
    3   0.25
    4  -4.10
    5   0.00
   Terms:
     'x0' (column 0)
     'x1' (column 1))

In [20]:
a,b,c,d = np.linalg.lstsq(X, y)

C:\Users\ashwi\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """Entry point for launching an IPython kernel.


In [21]:
print(a,b,c,d)

[[ 0.31290976]
 [-0.07910564]
 [-0.26546384]] [19.63791494] 3 [8.03737688 3.38335321 0.90895207]


In [22]:
X.design_info.column_names

['Intercept', 'x0', 'x1']

In [23]:
y,X = patsy.dmatrices('y ~ x0 + np.log(np.abs(x1)+1)+0',data)

In [24]:
X

DesignMatrix with shape (5, 2)
  x0  np.log(np.abs(x1) + 1)
   1                 0.00995
   2                 0.00995
   3                 0.22314
   4                 1.62924
   5                 0.00000
  Terms:
    'x0' (column 0)
    'np.log(np.abs(x1) + 1)' (column 1)

In [25]:
y, X = patsy.dmatrices('y ~ standardize(x0) + center(x1)', data)


In [26]:
X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

In [27]:
new_data = pd.DataFrame({
   ....:     'x0': [6, 7, 8, 9],
   ....:     'x1': [3.1, -0.5, 0, 2.3],
   ....:     'y': [1, 2, 3, 4]})
new_data

,x0,x1,y
0,6,3.1,1
1,7,-0.5,2
2,8,0.0,3
3,9,2.3,4


In [28]:
X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

In [29]:
new_y,new_x = patsy.build_design_matrices([y.design_info,X.design_info],new_data)

In [30]:
new_x

DesignMatrix with shape (4, 3)
  Intercept  standardize(x0)  center(x1)
          1          2.12132        3.87
          1          2.82843        0.27
          1          3.53553        0.77
          1          4.24264        3.07
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

In [31]:
y, X = patsy.dmatrices('y ~ I(x0 + x1)', data)

In [32]:
X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

In [33]:
data = pd.DataFrame({
   ....:     'key1': ['a', 'a', 'b', 'b', 'a', 'b', 'a', 'b'],
   ....:     'key2': [0, 1, 0, 1, 0, 1, 0, 0],
   ....:     'v1': [1, 2, 3, 4, 5, 6, 7, 8],
   ....:     'v2': [-1, 0, 2.5, -0.5, 4.0, -1.2, 0.2, -1.7]
   ....: })

In [34]:
data

,key1,key2,v1,v2
0,a,0,1,-1.0
1,a,1,2,0.0
2,b,0,3,2.5
3,b,1,4,-0.5
4,a,0,5,4.0
5,b,1,6,-1.2
6,a,0,7,0.2
7,b,0,8,-1.7


In [35]:
y,X = patsy.dmatrices('v2 ~ key1 + 0',data)

In [36]:
X

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

In [37]:
y,X = patsy.dmatrices('v2~C(key2)+0',data)

In [38]:
X

DesignMatrix with shape (8, 2)
  C(key2)[0]  C(key2)[1]
           1           0
           0           1
           1           0
           0           1
           1           0
           0           1
           1           0
           1           0
  Terms:
    'C(key2)' (columns 0:2)

In [39]:
data['key2'] = data['key2'].map({0:'Zero',1:'one'})

In [40]:
data

,key1,key2,v1,v2
0,a,Zero,1,-1.0
1,a,one,2,0.0
2,b,Zero,3,2.5
3,b,one,4,-0.5
4,a,Zero,5,4.0
5,b,one,6,-1.2
6,a,Zero,7,0.2
7,b,Zero,8,-1.7


In [41]:
y,X = patsy.dmatrices('v2 ~ key1+key2+key1:key2+0',data)

In [42]:
X

DesignMatrix with shape (8, 4)
  key1[a]  key1[b]  key2[T.one]  key1[T.b]:key2[T.one]
        1        0            0                      0
        1        0            1                      0
        0        1            0                      0
        0        1            1                      1
        1        0            0                      0
        0        1            1                      1
        1        0            0                      0
        0        1            0                      0
  Terms:
    'key1' (columns 0:2)
    'key2' (column 2)
    'key1:key2' (column 3)

In [43]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [44]:
def dnorm(mean, variance, size=1):
    if isinstance(size, int):
        size = size,
    return mean + np.sqrt(variance) * np.random.randn(*size)


In [55]:
# For reproducibility
np.random.seed(12345)

N = 100
X = np.c_[dnorm(0, 0.4, size=N),
          dnorm(0, 0.6, size=N),
          dnorm(0, 0.2, size=N)]
print(len(X))
eps = dnorm(0, 0.1, size=N)
beta = [0.1, 0.3, 0.5]

y = np.dot(X, beta) + eps
len(y)

100


100

In [48]:
X_model = sm.add_constant(X)
X_model[:5]

array([[ 1.        , -0.12946849, -1.21275292,  0.50422488],
       [ 1.        ,  0.30291036, -0.43574176, -0.25417986],
       [ 1.        , -0.32852189, -0.02530153,  0.13835097],
       [ 1.        , -0.35147471, -0.71960511, -0.25821463],
       [ 1.        ,  1.2432688 , -0.37379916, -0.52262905]])

In [49]:
model = sm.OLS(y,X)

In [51]:
result = model.fit()

In [52]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.430
Model:                            OLS   Adj. R-squared:                  0.413
Method:                 Least Squares   F-statistic:                     24.42
Date:                Fri, 29 Mar 2019   Prob (F-statistic):           7.44e-12
Time:                        14:18:22   Log-Likelihood:                -34.305
No. Observations:                 100   AIC:                             74.61
Df Residuals:                      97   BIC:                             82.42
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.1783      0.053      3.364      0.001       0.073       0.283
x2             0.2230      0.046      4.818      0.000       0.131       0.315
x3             0.5010      0.080      6.237      0.000       0.342       0.660
==============================================================================
Omnibus:                        4.662   Durbin-Watson:                   2.201
Prob(Omnibus):                  0.097   Jarque-Bera (JB):                4.098
Skew:                           0.481   Prob(JB):                        0.129
Kurtosis:                       3.243   Cond. No.                         1.74
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""